# SVM

### 기본 라이브러리 불러오기
[Step 1] 데이터 준비/ 기본 설정



In [3]:
### 기본 라이브러리 불러오기
import pandas as pd
import seaborn as sns

# load_dataset 함수를 사용하여 데이터프레임으로 변환
df = sns.load_dataset('titanic')

[Step 2] 데이터 탐색/ 전처리

In [4]:
# NaN값이 많은 deck 열을 삭제, embarked와 내용이 겹치는 embark_town 열을 삭제
rdf = df.drop(['deck', 'embark_town'], axis=1)  

# age 열에 나이 데이터가 없는 모든 행을 삭제 - age 열(891개 중 177개의 NaN 값)
rdf = rdf.dropna(subset=['age'], how='any', axis=0)  

# embarked 열의 NaN값을 승선도시 중에서 가장 많이 출현한 값으로 치환하기
most_freq = rdf['embarked'].value_counts(dropna=True).idxmax()   
rdf['embarked'].fillna(most_freq, inplace=True)

[Step 3] 분석에 사용할 속성을 선택

In [5]:
rdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 714 entries, 0 to 890
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   survived    714 non-null    int64   
 1   pclass      714 non-null    int64   
 2   sex         714 non-null    object  
 3   age         714 non-null    float64 
 4   sibsp       714 non-null    int64   
 5   parch       714 non-null    int64   
 6   fare        714 non-null    float64 
 7   embarked    714 non-null    object  
 8   class       714 non-null    category
 9   who         714 non-null    object  
 10  adult_male  714 non-null    bool    
 11  alive       714 non-null    object  
 12  alone       714 non-null    bool    
dtypes: bool(2), category(1), float64(2), int64(4), object(4)
memory usage: 63.6+ KB


In [6]:
# 분석에 활용할 열(속성)을 선택 
ndf = rdf[['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'embarked']]
ndf.head()

,survived,pclass,sex,age,sibsp,parch,embarked
0,0,3,male,22.0,1,0,S
1,1,1,female,38.0,1,0,C
2,1,3,female,26.0,0,0,S
3,1,1,female,35.0,1,0,S
4,0,3,male,35.0,0,0,S


In [7]:
# 원핫인코딩 - 범주형 데이터를 모형이 인식할 수 있도록 숫자형으로 변환
onehot_sex = pd.get_dummies(ndf['sex'])
ndf = pd.concat([ndf, onehot_sex], axis=1)
onehot_embarked = pd.get_dummies(ndf['embarked'], prefix='town')
ndf = pd.concat([ndf, onehot_embarked], axis=1)
ndf.drop(['sex', 'embarked'], axis=1, inplace=True)

In [8]:
ndf.head()

,survived,pclass,age,sibsp,parch,female,male,town_C,town_Q,town_S
0,0,3,22.0,1,0,0,1,0,0,1
1,1,1,38.0,1,0,1,0,1,0,0
2,1,3,26.0,0,0,1,0,0,0,1
3,1,1,35.0,1,0,1,0,0,0,1
4,0,3,35.0,0,0,0,1,0,0,1


[Step 4] 데이터셋 구분 - 훈련용(train data)/ 검증용(test data)

In [9]:
# 속성(변수) 선택
X=ndf[['pclass', 'age', 'sibsp', 'parch', 'female', 'male', 
       'town_C', 'town_Q', 'town_S']]  #독립 변수 X
y=ndf['survived']                      #종속 변수 Y

# 설명 변수 데이터를 정규화(normalization)
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)

In [10]:
# train data 와 test data로 구분(7:3 비율)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.3, 
                                                    random_state=10) 

print('train data 개수: ', X_train.shape)
print('test data 개수: ', X_test.shape)

train data 개수:  (499, 9)
test data 개수:  (215, 9)


[Step 5] SVM 분류 모형 - sklearn 사용

In [11]:
# sklearn 라이브러리에서 SVM 분류 모형 가져오기
from sklearn import svm
# 모형 객체 생성 (기본값 kernel='rbf' 적용)
svm_model = svm.SVC()
# train data를 가지고 모형 학습
svm_model.fit(X_train, y_train) 

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [12]:
# test data를 가지고 y_hat을 예측 (분류) 
y_hat = svm_model.predict(X_test)

In [13]:
import pandas as pd
df = pd.DataFrame({'y_hat':y_hat,'y':y_test})
df['차이'] = df['y_hat'] == df['y']
df.head()

,y_hat,y,차이
728,0,0,True
555,0,0,True
426,1,1,True
278,0,0,True
617,0,0,True


In [14]:
print('오답 수:',len(df) - sum(df['차이']))

오답 수: 40


In [15]:
# 모형 성능 평가 - Confusion Matrix 계산
from sklearn import metrics 
svm_matrix = metrics.confusion_matrix(y_test, y_hat) 
svm_matrix

array([[120,   5],
       [ 35,  55]])

In [ ]:
# 모형 성능 평가 - 평가지표 계산
svm_report = metrics.classification_report(y_test, y_hat)  
print(svm_report)

              precision    recall  f1-score   support

           0       0.77      0.96      0.86       125
           1       0.92      0.61      0.73        90

    accuracy                           0.81       215
   macro avg       0.85      0.79      0.80       215
weighted avg       0.83      0.81      0.81       215



### 붓꽃

In [ ]:
from sklearn.datasets import load_iris
iris_dataset = load_iris() # 붓꽃 데이터셋을 적재합니다. 
iris_df = pd.DataFrame(iris_dataset['data'],
                       columns=iris_dataset['feature_names'])

In [ ]:
iris_df['target_names']=iris_dataset['target']

In [ ]:
iris_df['sepal length (cm)'] = (iris_df['sepal length (cm)'] - iris_df['sepal length (cm)'].mean())/iris_df['sepal length (cm)'].std()
iris_df['sepal width (cm)'] = (iris_df['sepal width (cm)'] - iris_df['sepal width (cm)'].mean())/iris_df['sepal width (cm)'].std()
iris_df['petal length (cm)'] =(iris_df['petal length (cm)'] - iris_df['petal length (cm)'].mean())/iris_df['petal length (cm)'].std()
iris_df['petal width (cm)'] = (iris_df['petal width (cm)'] - iris_df['petal width (cm)'].mean())/iris_df['petal width (cm)'].std()
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target_names
0,-0.897674,1.015602,-1.335752,-1.311052,0
1,-1.139200,-0.131539,-1.335752,-1.311052,0
2,-1.380727,0.327318,-1.392399,-1.311052,0
3,-1.501490,0.097889,-1.279104,-1.311052,0
4,-1.018437,1.245030,-1.335752,-1.311052,0


In [ ]:
iris_df.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target_names
count,1.500000e+02,1.500000e+02,1.500000e+02,1.500000e+02,150.000000
mean,-1.457168e-15,-1.638319e-15,-1.292300e-15,-5.543714e-16,1.000000
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.819232
min,-1.863780e+00,-2.425820e+00,-1.562342e+00,-1.442245e+00,0.000000
25%,-8.976739e-01,-5.903951e-01,-1.222456e+00,-1.179859e+00,0.000000
50%,-5.233076e-02,-1.315388e-01,3.353541e-01,1.320673e-01,1.000000
75%,6.722490e-01,5.567457e-01,7.602115e-01,7.880307e-01,2.000000
max,2.483699e+00,3.080455e+00,1.779869e+00,1.706379e+00,2.000000


In [ ]:
# 속성(변수) 선택
X = iris_df[list(iris_df.columns)[0:4]]  #독립 변수 X
y = iris_df['target_names']  

In [ ]:
# train data 와 test data로 구분(7:3 비율)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, 
                                                    random_state=10) 

print('train data 개수: ', X_train.shape)
print('test data 개수: ', X_test.shape)

train data 개수:  (105, 4)
test data 개수:  (45, 4)


In [ ]:
# sklearn 라이브러리에서 SVM 분류 모형 가져오기
from sklearn import svm

# 모형 객체 생성
svm_model = svm.SVC()

# train data를 가지고 모형 학습
svm_model.fit(X_train, y_train) 

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
# test data를 가지고 y_hat을 예측 (분류) 
y_hat = svm_model.predict(X_test)

In [ ]:
# 모형 성능 평가 - Confusion Matrix 계산
from sklearn import metrics 
svm_matrix = metrics.confusion_matrix(y_test, y_hat) 
svm_matrix

array([[14,  0,  0],
       [ 0, 17,  0],
       [ 0,  0, 14]])

In [ ]:
# 모형 성능 평가 - 평가지표 계산
svm_report = metrics.classification_report(y_test, y_hat)  
print(svm_report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        14
           1       1.00      1.00      1.00        17
           2       1.00      1.00      1.00        14

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45

